In [8]:
import ccxt
import pandas as pd
import time

def fetch_binance_ohlcv_batched(symbol, timeframe, start_date, end_date, output_file, max_limit=500):
    # Initialize Binance exchange
    exchange = ccxt.binance()

    # Convert start and end dates to milliseconds
    since = int(pd.Timestamp(start_date).timestamp() * 1000)
    end_timestamp = int(pd.Timestamp(end_date).timestamp() * 1000)

    # Write the CSV header
    with open(output_file, "w") as f:
        f.write("timestamp,open,high,low,close,volume\n")
    
    total_rows = 0  # Counter for total rows fetched

    # Fetch data in batches
    while since < end_timestamp:
        try:
            # Fetch OHLCV data
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=max_limit)
            if not ohlcv:
                print("No more data to fetch.")
                break

            # Convert to DataFrame
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

            # Append to CSV
            df.to_csv(output_file, mode='a', header=False, index=False)

            # Update the `since` parameter for the next batch
            since = ohlcv[-1][0] + 1  # Move to the next timestamp
            total_rows += len(df)

            print(f"Fetched {len(df)} rows. Total rows so far: {total_rows}")

            # Sleep to respect API rate limits
            time.sleep(0.2)

        except Exception as e:
            print(f"Error fetching data: {e}")
            time.sleep(1)  # Retry delay

    print(f"Data fetching completed. Total rows: {total_rows}. Data saved to {output_file}")



In [6]:
fetch_binance_ohlcv_batched("ETH/USDT", "1h", "2020-01-01", "2025-03-13", "ETH_usdt_1h_2020JAN01_2025MAR13.csv")
fetch_binance_ohlcv_batched("BTC/USDT", "1h", "2020-01-01", "2025-03-13", "BTC_usdt_1h_2020JAN01_2025MAR13.csv")
fetch_binance_ohlcv_batched("SOL/USDT", "1h", "2020-08-01", "2025-03-13", "SOL_usdt_1h_2020AUG11_2025MAR13.csv")

KeyboardInterrupt: 

In [10]:
# check time frame
df1 = pd.read_csv("ETH_usdt_1h_2020JAN01_2025MAR14.csv")
df2 = pd.read_csv("BTC_usdt_1h_2020JAN01_2025MAR14.csv")
df3 = pd.read_csv("SOL_usdt_1h_2020AUG11_2025MAR14.csv")
# check time frame match
print(df1['timestamp'].min(), df1['timestamp'].max())
print(df2['timestamp'].min(), df2['timestamp'].max())
print(df3['timestamp'].min(), df3['timestamp'].max())

2020-01-01 00:00:00 2025-03-14 10:00:00
2020-01-01 00:00:00 2025-03-14 10:00:00
2020-08-11 06:00:00 2025-03-14 10:00:00
